In [1]:
import os; os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2

In [184]:
import numpy as np
import matplotlib.pyplot as plt
from urbanstats.geometry.shapefiles.shapefiles_list import shapefiles
import geopandas as gpd
import pandas as pd

import shapely.geometry

In [22]:
cities = shapefiles["cities"].load_file()
nbhds = shapefiles["neighborhoods"].load_file()

  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/106 [00:00<?, ?it/s]

In [7]:
top, left = 42.42092591917809, -71.30407957473298
bottom, right = 42.192030016147555, -70.96676497377845

In [84]:
def pull_shapefile(sh):
    return sh[sh.bounds.apply(lambda x: x.maxx > left and x.minx < right and x.maxy > bottom and x.miny < top, axis=1)].copy()

In [86]:
cities_in_mbta_hiding_zones = {
    'Arlington CDP, Massachusetts, USA',
    'Boston city, Massachusetts, USA',
    'Braintree Town city, Massachusetts, USA',
    'Brookline CDP, Massachusetts, USA',
    'Cambridge city, Massachusetts, USA',
    'Chelsea city, Massachusetts, USA',
    'Everett city, Massachusetts, USA',
    'Malden city, Massachusetts, USA',
    'Medford city, Massachusetts, USA',
    'Milton CDP, Massachusetts, USA',
    'Newton city, Massachusetts, USA',
    'Quincy city, Massachusetts, USA',
    'Revere city, Massachusetts, USA',
    'Somerville city, Massachusetts, USA',
}

In [188]:
rc = cities[cities.longname.isin(cities_in_mbta_hiding_zones)].copy()
rc["city"] = rc.longname.apply(lambda x: x.split()[0])
rc["name"] = rc.city
boston = rc[rc.city == "Boston"].geometry.iloc[0]
rc = rc[rc.city != "Boston"].copy()
rn = pull_shapefile(nbhds)
rn = rn[rn.shortname.apply(lambda x: x.split(", ")[-1]) == "Boston"].copy()
rn["city"] = "Boston"
rn["name"] = rn.shortname.apply(lambda x: x.split(", ")[0])
rn["geometry"] = rn.geometry.apply(lambda x: shapely.intersection(x, boston))

In [190]:
result = gpd.GeoDataFrame(pd.concat([rc, rn]))

In [193]:
with open("/home/kavi/temp/shapes.geojson", "w") as f:
    f.write(result.to_json())